# N-gram Language Models - Practice with Real Data

**Course:** Natural Language Processing  
**Topic:** Building N-gram Models with HuggingFace Datasets  

In this notebook, you will:
1. Load real text corpora from HuggingFace
2. Implement n-gram language models from scratch
3. Train and evaluate your models
4. Generate text and compare different approaches

## Setup

In [2]:
# Install required packages
!pip install datasets -q

import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
import random
from datasets import load_dataset
from tqdm import tqdm

# Set random seed
np.random.seed(42)
random.seed(42)

print("✓ Setup complete!")

✓ Setup complete!


## Part 1: Load Dataset from HuggingFace

We'll use the **WikiText-2** dataset, which contains good quality text from Wikipedia articles.

In [3]:
# Load WikiText-2 dataset
print("Loading WikiText-2 dataset...")
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

print("\nDataset structure:")
print(dataset)

# Explore the data
print("\nSample from training set:")
print(dataset['train'][0])
print("\n" + "="*60)
print(dataset['train'][1])

Loading WikiText-2 dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Dataset structure:
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

Sample from training set:
{'text': ''}

{'text': ' = Valkyria Chronicles III = \n'}


### Exercise 1.1: Data Exploration

Answer these questions:
1. How many examples are in train/validation/test splits?
2. What does each example look like?
3. How many empty or very short texts are there?

In [4]:
train_size = len(dataset['train'])
valid_size = len(dataset['validation'])
test_size = len(dataset['test'])

print(f"Train size: {train_size}")
print(f"Validation size: {valid_size}")
print(f"Test size: {test_size}")

# Question 2: Look at a few examples
print("\nFirst 5 examples from training set:")
for i in range(5):
    print(f"Example {i+1}: {dataset['train'][i]['text']}")


# Question 3: Check for empty or short texts
print("\nChecking for empty or short texts:")
empty_or_short_texts = 0
for item in dataset['train']:
    text = item['text'].strip()
    if not text or len(text.split()) < 5:
        empty_or_short_texts += 1

print(f"Number of empty or short texts (less than 5 words) in training set: {empty_or_short_texts}")

Train size: 36718
Validation size: 3760
Test size: 4358

First 5 examples from training set:
Example 1: 
Example 2:  = Valkyria Chronicles III = 

Example 3: 
Example 4:  Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 

Example 5:  The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II .

## Part 2: Data Preprocessing

### Exercise 2.1: Implement Text Preprocessing

In [5]:
def preprocess_text(text: str, add_start_end: bool = True) -> List[str]:
    """
    Preprocess text:
    - Convert to lowercase
    - Split into tokens (simple whitespace tokenization)
    - Add <s> and </s> tokens if requested

    Args:
        text: Input text
        add_start_end: Whether to add start/end tokens

    Returns:
        List of tokens
    """
    # Step 1: Convert to lowercase
    text = text.lower()

    # Step 2: Split on whitespace
    tokens = text.split()

    # Step 3: Add <s> and </s> if requested
    if add_start_end:
        tokens = ['<s>'] + tokens + ['</s>']

    return tokens

# Test your function
test_text = "Hello World! This is a test."
tokens = preprocess_text(test_text, add_start_end=True)
print(f"Input: {test_text}")
print(f"Expected: ['<s>', 'hello', 'world!', 'this', 'is', 'a', 'test.', '</s>']")
print(f"Your output: {tokens}")

Input: Hello World! This is a test.
Expected: ['<s>', 'hello', 'world!', 'this', 'is', 'a', 'test.', '</s>']
Your output: ['<s>', 'hello', 'world!', 'this', 'is', 'a', 'test.', '</s>']


### Exercise 2.2: Prepare Training Corpus

In [6]:
def prepare_corpus(dataset_split, max_examples: int = None) -> List[List[str]]:
    """
    Prepare corpus from dataset split.

    Args:
        dataset_split: HuggingFace dataset split
        max_examples: Maximum number of examples to use (None = all)

    Returns:
        List of tokenized sentences
    """
    corpus = []
    count = 0
    for item in tqdm(dataset_split, desc="Processing"):
        text = item['text'].strip()
        if not text:
            continue

        # Preprocess each text
        tokens = preprocess_text(text, add_start_end=True)

        # Add to corpus only if result has more than 2 tokens (s, /s)
        if len(tokens) > 2:
            corpus.append(tokens)
            count += 1

        # Stop at max_examples if specified
        if max_examples and count >= max_examples:
            break

    return corpus

# Prepare training corpus (use subset for speed)
print("Preparing training corpus...")
train_corpus = prepare_corpus(dataset['train'], max_examples=1000)

print(f"\nCorpus size: {len(train_corpus)} sentences")
print(f"First sentence: {train_corpus[0][:10]}...")  # Show first 10 tokens

Preparing training corpus...


Processing:   4%|▍         | 1538/36718 [00:00<00:01, 33375.27it/s]


Corpus size: 1000 sentences
First sentence: ['<s>', '=', 'valkyria', 'chronicles', 'iii', '=', '</s>']...


## Part 3: Build Vocabulary

### Exercise 3.1: Build Vocabulary

In [7]:
def build_vocabulary(corpus: List[List[str]]) -> set:
    """
    Build vocabulary from corpus.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Set of unique tokens
    """
    # Hint: Create a set and add all tokens from all sentences
    vocabulary = set()
    for sentence in corpus:
        for token in sentence:
            vocabulary.add(token)
    return vocabulary

# Build vocabulary
vocab = build_vocabulary(train_corpus)
print(f"Vocabulary size: {len(vocab)}")
print(f"Sample words: {list(vocab)[:20]}")

Vocabulary size: 10690
Sample words: ['loaded', 'disapproved', 'genera', 'roman', 'perfectly', 'icon', 'triads', 'bag', 'large', 'masted', 'roberts', 'radetzky', 'interact', 'combat', 'dance', 'wanted', 'avalanche', 'def', 'understanding', 'brick']


## Part 4: Implement N-gram Counting

### Exercise 4.1: Count Unigrams

In [8]:
def count_unigrams(corpus: List[List[str]]) -> Counter:
    """
    Count unigram frequencies.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Counter with unigram counts
    """
    unigram_counts = Counter()

    # TODO: Implement this function
    # Iterate through corpus and count each token
    for sentence in corpus:
        unigram_counts.update(sentence)

    return unigram_counts

# Test
unigram_counts = count_unigrams(train_corpus)
print(f"Total unigrams: {sum(unigram_counts.values())}")
print(f"Unique unigrams: {len(unigram_counts)}")
print(f"\nMost common unigrams:")
print(unigram_counts.most_common(20))

Total unigrams: 90023
Unique unigrams: 10690

Most common unigrams:
[('the', 5757), (',', 4468), ('.', 3203), ('of', 2568), ('and', 2237), ('in', 1927), ('to', 1655), ('a', 1574), ('=', 1191), ('<s>', 1000), ('</s>', 1000), ('"', 961), ('was', 865), ('@-@', 782), ('with', 697), ('as', 663), ('for', 662), ('that', 623), ("'s", 606), ('on', 594)]


### Exercise 4.2: Count Bigrams

In [9]:
def count_bigrams(corpus: List[List[str]]) -> Counter:
    """
    Count bigram frequencies.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Counter with bigram counts (bigrams as tuples)
    """
    bigram_counts = Counter()

    # For each sentence, create pairs of consecutive words
    # Store as tuples: (word1, word2)
    for sentence in corpus:
        for i in range(len(sentence) - 1):
            bigram_counts[(sentence[i], sentence[i+1])] += 1

    return bigram_counts

# Test
bigram_counts = count_bigrams(train_corpus)
print(f"Total bigrams: {sum(bigram_counts.values())}")
print(f"Unique bigrams: {len(bigram_counts)}")
print(f"\nMost common bigrams:")
print(bigram_counts.most_common(20))

Total bigrams: 89023
Unique bigrams: 50573

Most common bigrams:
[(('of', 'the'), 837), (('=', '='), 656), ((',', 'and'), 607), (('.', '</s>'), 578), (('.', 'the'), 526), (('in', 'the'), 498), ((',', 'the'), 316), (('to', 'the'), 268), (('<s>', '='), 264), (('=', '</s>'), 264), (('on', 'the'), 199), (('<s>', 'the'), 181), (('for', 'the'), 178), (('and', 'the'), 177), (('with', 'the'), 170), (('at', 'the'), 157), (('.', 'in'), 156), (('.', '"'), 130), ((',', 'a'), 127), ((',', 'which'), 124)]


### Exercise 4.3: Count Trigrams

In [10]:
def count_trigrams(corpus: List[List[str]]) -> Counter:
    """
    Count trigram frequencies.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Counter with trigram counts (trigrams as tuples)
    """
    trigram_counts = Counter()

    # For each sentence, create triples of consecutive words
    # Store as tuples: (word1, word2, word3)
    for sentence in corpus:
        for i in range(len(sentence) - 2):
            trigram_counts[(sentence[i], sentence[i+1], sentence[i+2])] += 1

    return trigram_counts

# Test
trigram_counts = count_trigrams(train_corpus)
print(f"Total trigrams: {sum(trigram_counts.values())}")
print(f"Unique trigrams: {len(trigram_counts)}")
print(f"\nMost common trigrams:")
print(trigram_counts.most_common(20))

Total trigrams: 88023
Unique trigrams: 76398

Most common trigrams:
[(('<s>', '=', '='), 230), (('=', '=', '</s>'), 230), (('=', '=', '='), 196), ((',', 'and', 'the'), 65), (('the', 'blue', 'jackets'), 39), (('one', 'of', 'the'), 37), (('.', '"', '</s>'), 34), ((';', 'blackie', ','), 30), (('as', 'well', 'as'), 29), (('"', '.', '</s>'), 27), ((',', 'such', 'as'), 27), (('the', 'gold', 'dollar'), 27), (('strapping', 'young', 'lad'), 26), (('the', 'united', 'states'), 25), (('.', 'she', 'was'), 24), (('.', 'it', 'was'), 22), ((',', 'and', 'a'), 22), (('.', 'in', 'the'), 22), (('south', 'of', 'heaven'), 22), (('in', 'the', 'uk'), 21)]


## Part 5: Implement Language Models

### Exercise 5.1: Unigram Model

In [11]:
class UnigramModel:
    """Unigram language model."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.total_words = sum(self.unigram_counts.values())

    def probability(self, word: str) -> float:
        """Calculate P(word)."""
        # P(word) = C(word) / N
        count = self.unigram_counts.get(word, 0)
        return count / self.total_words if self.total_words > 0 else 0.0

    def log_probability(self, word: str) -> float:
        """Calculate log P(word)."""
        # Handle zero probability by returning float('-inf')
        prob = self.probability(word)
        return np.log(prob) if prob > 0 else float('-inf')

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # Sum of log probabilities of all words
        log_prob_sum = 0.0
        for word in sentence:
            log_prob_sum += self.log_probability(word)
        return log_prob_sum

# Test your model
unigram_model = UnigramModel(train_corpus)
test_words = ["the", "of", "and", "<s>", "</s>"]
print("Unigram Probabilities:")
for word in test_words:
    prob = unigram_model.probability(word)
    print(f"  P({word}) = {prob:.6f}")

Unigram Probabilities:
  P(the) = 0.063950
  P(of) = 0.028526
  P(and) = 0.024849
  P(<s>) = 0.011108
  P(</s>) = 0.011108


### Exercise 5.2: Bigram Model

In [12]:
class BigramModel:
    """Bigram language model."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.bigram_counts = count_bigrams(corpus)

    def probability(self, word: str, previous_word: str) -> float:
        """Calculate P(word | previous_word)."""
        # P(wi | wi-1) = C(wi-1, wi) / C(wi-1)
        count_prev_word = self.unigram_counts.get(previous_word, 0)
        if count_prev_word == 0:
            return 0.0 # Cannot divide by zero if previous word never appeared

        count_bigram = self.bigram_counts.get((previous_word, word), 0)
        return count_bigram / count_prev_word

    def log_probability(self, word: str, previous_word: str) -> float:
        """Calculate log P(word | previous_word)."""
        prob = self.probability(word, previous_word)
        return np.log(prob) if prob > 0 else float('-inf')

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # Sum log probabilities: log P(w2|w1) + log P(w3|w2) + ...
        # Assuming sentence starts with <s>
        log_prob_sum = 0.0
        if len(sentence) < 2:
            # A sentence with 0 or 1 token (e.g., just '<s>' or '<s> </s>') might not make sense for bigrams
            # For simplicity, we'll return -inf or a reasonable default for such cases
            # If it's just '<s>', its probability is 1 (log_prob = 0) in the context of start token itself
            # However, typically bigram sentences have at least <s> and another word.
            return 0.0 # log P(<s>) is effectively 0 in this context

        # First term P(w1|<s>) for sentence[1] given sentence[0] (which is <s>)
        # The loop naturally handles this by starting from the second word
        for i in range(1, len(sentence)): # Start from the second word (index 1)
            log_prob_sum += self.log_probability(sentence[i], sentence[i-1])
        return log_prob_sum

# Test your model
bigram_model = BigramModel(train_corpus)
test_bigrams = [("<s>", "the"), ("the", "and"), ("of", "the")]
print("Bigram Probabilities:")
for w1, w2 in test_bigrams:
    prob = bigram_model.probability(w2, w1)
    print(f"  P({w2} | {w1}) = {prob:.6f}")

Bigram Probabilities:
  P(the | <s>) = 0.181000
  P(and | the) = 0.000000
  P(the | of) = 0.325935


### Exercise 5.3: Trigram Model

In [13]:
class TrigramModel:
    """Trigram language model."""

    def __init__(self, corpus: List[List[str]]):
        self.bigram_counts = count_bigrams(corpus)
        self.trigram_counts = count_trigrams(corpus)

    def probability(self, word: str, prev_word1: str, prev_word2: str) -> float:
        """Calculate P(word | prev_word2, prev_word1)."""
        # P(wi | wi-2, wi-1) = C(wi-2, wi-1, wi) / C(wi-2, wi-1)
        context_bigram_count = self.bigram_counts.get((prev_word2, prev_word1), 0)
        if context_bigram_count == 0:
            return 0.0 # Cannot divide by zero if context bigram never appeared

        trigram_count = self.trigram_counts.get((prev_word2, prev_word1, word), 0)
        return trigram_count / context_bigram_count

    def log_probability(self, word: str, prev_word1: str, prev_word2: str) -> float:
        """Calculate log P(word | prev_word2, prev_word1)."""
        prob = self.probability(word, prev_word1, prev_word2)
        return np.log(prob) if prob > 0 else float('-inf')

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # Sum log probabilities: log P(w3|w1,w2) + log P(w4|w2,w3) + ...
        # Assuming sentence starts with <s>
        log_prob_sum = 0.0
        if len(sentence) < 3:
            # A sentence with less than 3 tokens (e.g., '<s>', '<s> w1 </s>')
            # For simplicity, we'll return 0.0 for such cases, as effectively the initial <s>'s have prob 1
            return 0.0

        # The loop naturally handles this by starting from the third word
        for i in range(2, len(sentence)): # Start from the third word (index 2)
            log_prob_sum += self.log_probability(sentence[i], sentence[i-1], sentence[i-2])
        return log_prob_sum

# Test your model
trigram_model = TrigramModel(train_corpus)
if len(train_corpus[0]) >= 3:
    w1, w2, w3 = train_corpus[0][:3]
    prob = trigram_model.probability(w3, w2, w1)
    print(f"P({w3} | {w1}, {w2}) = {prob:.6f}")
else:
    print("First sentence in train_corpus is too short for trigram test.")

P(valkyria | <s>, =) = 0.003788


## Part 6: Model Evaluation - Perplexity

### Exercise 6.1: Implement Perplexity Calculation

In [14]:
def calculate_perplexity(model, test_corpus: List[List[str]], model_type: str = "bigram") -> float:
    """
    Calculate perplexity for a language model.

    Perplexity = exp(-1/N * sum(log P(wi | context)))

    Args:
        model: Language model with sentence_log_probability method
        test_corpus: List of test sentences
        model_type: 'unigram', 'bigram', or 'trigram'

    Returns:
        Perplexity score
    """
    total_log_prob = 0.0
    total_words = 0

    for sentence in tqdm(test_corpus, desc="Calculating perplexity"):
        if model_type == "unigram":
            total_log_prob += model.sentence_log_probability(sentence)
            total_words += len(sentence) # All words contribute to N
        elif model_type == "bigram":
            total_log_prob += model.sentence_log_probability(sentence)
            # For bigram, the first word (<s>) does not have a previous word for a conditional probability
            # So N is effectively len(sentence) - 1 if <s> is always the first token
            # But sentence_log_probability already skips the first word if it calculates P(w_i | w_{i-1})
            # So total_words should count all words that *have* a preceding word.
            # If sentence_log_probability sums log P(wi | context) for i=1 to N-1, then N is N-1 tokens.
            total_words += len(sentence) - 1 if len(sentence) > 1 else 0
        elif model_type == "trigram":
            total_log_prob += model.sentence_log_probability(sentence)
            # For trigram, the first two words (<s> and w1) do not have enough preceding words
            # So N is effectively len(sentence) - 2 if <s> is w0 and w1 is the second token
            total_words += len(sentence) - 2 if len(sentence) > 2 else 0
        else:
            raise ValueError("Invalid model_type. Must be 'unigram', 'bigram', or 'trigram'.")

    if total_words == 0:
        return float('inf') # Avoid division by zero, perplexity is infinite if no words counted

    # Average log probability
    avg_log_prob = total_log_prob / total_words

    # Perplexity = exp(-avg_log_prob)
    return np.exp(-avg_log_prob)

# Prepare test corpus
print("Preparing test corpus...")
test_corpus = prepare_corpus(dataset['validation'], max_examples=100)
print(f"Test corpus size: {len(test_corpus)} sentences")

# Calculate perplexities
print("\nCalculating perplexities...")
unigram_perplexity = calculate_perplexity(unigram_model, test_corpus, model_type="unigram")
bigram_perplexity = calculate_perplexity(bigram_model, test_corpus, model_type="bigram")
trigram_perplexity = calculate_perplexity(trigram_model, test_corpus, model_type="trigram")
smoothed_bigram_perplexity = calculate_perplexity(smoothed_model, test_corpus, model_type="bigram")

print(f"Unigram Model Perplexity: {unigram_perplexity:.2f}")
print(f"Bigram Model Perplexity: {bigram_perplexity:.2f}")
print(f"Trigram Model Perplexity: {trigram_perplexity:.2f}")
print(f"Smoothed Bigram Model Perplexity: {smoothed_bigram_perplexity:.2f}")

Preparing test corpus...


Processing:   4%|▍         | 156/3760 [00:00<00:00, 19258.61it/s]


Test corpus size: 100 sentences

Calculating perplexities...


Calculating perplexity: 100%|██████████| 100/100 [00:00<00:00, 5048.27it/s]


NameError: name 'smoothed_model' is not defined

## Part 7: Text Generation

### Exercise 7.1: Implement Text Generator

In [ ]:
class BigramGenerator:
    """Generate text using bigram model."""

    def __init__(self, corpus: List[List[str]]):
        self.bigram_model = BigramModelSmoothed(corpus) # Use smoothed model for generation
        self.unigram_counts = count_unigrams(corpus)
        self.bigram_counts = count_bigrams(corpus)
        self.vocab = build_vocabulary(corpus)
        self.next_word_probs = self._build_next_word_distribution()

    def _build_next_word_distribution(self) -> Dict[str, Tuple[List[str], List[float]]]:
        """Build distribution over next words for each word."""
        distribution = defaultdict(lambda: ([], []))
        for prev_word in self.vocab:
            possible_next_words = []
            probabilities = []

            # Iterate through all words in vocab to find possible next words and their probabilities
            for next_word in self.vocab:
                prob = self.bigram_model.probability(next_word, prev_word)
                if prob > 0: # Only consider words with non-zero probability
                    possible_next_words.append(next_word)
                    probabilities.append(prob)

            if possible_next_words: # Normalize probabilities if not empty
                total_prob = sum(probabilities)
                probabilities = [p / total_prob for p in probabilities]
                distribution[prev_word] = (possible_next_words, probabilities)

        # Ensure <s> has a distribution
        if '<s>' not in distribution:
            # If '<s>' doesn't have any observed next words, create a dummy distribution
            # to allow generation to start, or handle it as an edge case.
            # For now, let's assume '<s>' always has some observed next words in the corpus.
            # If it truly doesn't, this part might need more robust handling.
            pass # This case should ideally not happen if <s> is used correctly

        return distribution

    def generate(self, max_length: int = 20) -> List[str]:
        """Generate a sentence."""
        sentence = ['<s>']
        current_word = '<s>'

        for _ in range(max_length - 1): # -1 because <s> is already in sentence
            if current_word not in self.next_word_probs:
                # If current_word has no known next word distribution, stop
                break

            possible_next_words, probabilities = self.next_word_probs[current_word]

            if not possible_next_words: # No possible next words
                break

            next_word = np.random.choice(possible_next_words, p=probabilities)
            sentence.append(next_word)
            current_word = next_word

            if current_word == '</s>':
                break

        # Ensure sentence ends with </s> if not already there and max_length not reached
        if sentence[-1] != '</s>' and len(sentence) < max_length:
            sentence.append('</s>')

        return sentence

# Test generator
generator = BigramGenerator(train_corpus)
print("Generated Sentences:")
print("=" * 60)
for i in range(5): # Generate 5 sentences for testing
    sentence = generator.generate(max_length=20)
    print(f"{i+1}. {' '.join(sentence)}")

## Part 8: Smoothing

### Exercise 8.1: Implement Add-One Smoothing

In [ ]:
class BigramModelSmoothed:
    """Bigram model with add-one smoothing."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.bigram_counts = count_bigrams(corpus)
        self.vocab = build_vocabulary(corpus)
        self.vocab_size = len(self.vocab)

    def probability(self, word: str, previous_word: str) -> float:
        """Calculate P(word | previous_word) with add-one smoothing.

        Formula: P(wi | wi-1) = (C(wi-1, wi) + 1) / (C(wi-1) + V)
        """
        # C(wi-1, wi) + 1
        numerator = self.bigram_counts.get((previous_word, word), 0) + 1
        # C(wi-1) + V
        denominator = self.unigram_counts.get(previous_word, 0) + self.vocab_size

        return numerator / denominator if denominator > 0 else 0.0

    def log_probability(self, word: str, previous_word: str) -> float:
        """Calculate log P(word | previous_word)."""
        prob = self.probability(word, previous_word)
        return np.log(prob) if prob > 0 else float('-inf')

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        log_prob_sum = 0.0
        if len(sentence) < 2:
            return 0.0

        for i in range(1, len(sentence)): # Start from the second word (index 1)
            log_prob_sum += self.log_probability(sentence[i], sentence[i-1])
        return log_prob_sum

# Compare with unsmoothed model
smoothed_model = BigramModelSmoothed(train_corpus)

# Test on unseen bigrams
test_unseen = [("<s>", "quantum"), ("the", "xylophone"), ("amazing", "unicorn")]
print("Comparison: Unsmoothed vs Smoothed")
print("=" * 60)
for w1, w2 in test_unseen:
    unsmoothed = bigram_model.probability(w2, w1)
    smoothed = smoothed_model.probability(w2, w1)
    print(f"P({w2} | {w1}):")
    print(f"  Unsmoothed: {unsmoothed:.8f}")
    print(f"  Smoothed:   {smoothed:.8f}")
    print()

## Part 9: Analysis and Comparison

### Exercise 9.1: Compare All Models

In [ ]:
# TODO: Create a comparison table showing:
# 1. Number of parameters (unique n-grams)
# 2. Training perplexity
# 3. Test perplexity
# 4. Sample generated sentences

import pandas as pd

# Calculate training perplexities for the models
train_unigram_perplexity = calculate_perplexity(unigram_model, train_corpus, model_type="unigram")
train_bigram_perplexity = calculate_perplexity(bigram_model, train_corpus, model_type="bigram")
train_trigram_perplexity = calculate_perplexity(trigram_model, train_corpus, model_type="trigram")
train_smoothed_bigram_perplexity = calculate_perplexity(smoothed_model, train_corpus, model_type="bigram")

comparison_data = {
    'Model': ['Unigram', 'Bigram', 'Trigram', 'Bigram (Smoothed)'],
    'Parameters': [
        len(unigram_counts),
        len(bigram_counts),
        len(trigram_counts),
        len(bigram_counts) # Smoothed Bigram has same number of parameters as unsmoothed
    ],
    'Train Perplexity': [
        train_unigram_perplexity,
        train_bigram_perplexity,
        train_trigram_perplexity,
        train_smoothed_bigram_perplexity
    ],
    'Test Perplexity': [
        unigram_perplexity,
        bigram_perplexity,
        trigram_perplexity,
        smoothed_bigram_perplexity
    ]
}

df = pd.DataFrame(comparison_data)
print("\nModel Comparison:")
print(df.to_string(index=False))

### Exercise 9.2: Visualize Results

In [ ]:
# TODO: Create visualizations:
# 1. Bar chart of perplexities
# 2. Top-20 most common words
# 3. Top-20 most common bigrams

# Example for most common words:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Most common unigrams
unigram_df = pd.DataFrame(unigram_counts.most_common(20), columns=['Word', 'Count'])
axes[0].bar(unigram_df['Word'], unigram_df['Count'])
axes[0].set_title('Top 20 Most Common Unigrams')
axes[0].set_xlabel('Word')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Plot 2: Most common bigrams
bigram_df = pd.DataFrame(bigram_counts.most_common(20), columns=['Bigram', 'Count'])
bigram_labels = [f'{w1} {w2}' for w1, w2 in bigram_df['Bigram']]
axes[1].bar(bigram_labels, bigram_df['Count'])
axes[1].set_title('Top 20 Most Common Bigrams')
axes[1].set_xlabel('Bigram')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()


## Part 10: Bonus Challenges

### Challenge 1: Try Different Datasets

Load a different dataset from HuggingFace and train your models on it.

Suggestions:
- `"wikitext"` (different versions)
- `"bookcorpus"`
- `"ptb_text_only"` (Penn Treebank)
- `"imdb"` (movie reviews)

In [ ]:
# TODO: YOUR CODE HERE
# Try loading and training on a different dataset

### Challenge 2: Implement Better Smoothing

Implement add-k smoothing (where k < 1) or interpolation smoothing.

In [ ]:
# TODO: YOUR CODE HERE

### Challenge 3: Build a Simple Autocomplete System

Given a partial sentence, suggest the top-k most likely next words.

In [ ]:
def autocomplete(partial_sentence: str, model: BigramModel, k: int = 5) -> List[Tuple[str, float]]:
    """
    Suggest next words for autocomplete.

    Args:
        partial_sentence: Incomplete sentence
        model: Trained bigram model
        k: Number of suggestions

    Returns:
        List of (word, probability) tuples
    """
    # TODO: Implement this
    pass

# Test
test_partial = "the cat sat on the"
suggestions = autocomplete(test_partial, bigram_model, k=5)
print(f"Suggestions for: '{test_partial}'")
for word, prob in suggestions:
    print(f"  {word}: {prob:.4f}")

# Task
Load the 'imdb' dataset from HuggingFace, using the 'train' split for training and 'test' split for evaluation.

## Load New Dataset

### Subtask:
Load the 'imdb' dataset from HuggingFace. We will specifically use the 'train' split for training and 'test' split for evaluation.


**Reasoning**:
Load the 'imdb' dataset using `load_dataset`, then print its structure and a sample from the training split to inspect its contents as per the instructions.



In [ ]:
print("Loading IMDb dataset...")
imdb_dataset = load_dataset("imdb")

print("\nDataset structure:")
print(imdb_dataset)

# Explore the data
print("\nSample from training set:")
print(imdb_dataset['train'][0])

## Prepare New Training Corpus

### Subtask:
Apply the `prepare_corpus` function to the 'train' split of the new 'imdb' dataset, using a smaller `max_examples` for demonstration.


**Reasoning**:
To apply the `prepare_corpus` function to the 'imdb' dataset's 'train' split with a specified `max_examples` limit, I need to call the function and then print the size and a sample of the resulting corpus for verification.



In [ ]:
print("Preparing IMDb training corpus...")
imdb_train_corpus = prepare_corpus(imdb_dataset['train'], max_examples=1000)

print(f"\nIMDb Corpus size: {len(imdb_train_corpus)} sentences")
print(f"First IMDb sentence: {imdb_train_corpus[0][:10]}...")

**Reasoning**:
The previous code block successfully prepared the IMDb training corpus. The next logical step in building an N-gram model is to build the vocabulary from this new corpus, following the original notebook's structure.



In [ ]:
print("Building vocabulary for IMDb corpus...")
imdb_vocab = build_vocabulary(imdb_train_corpus)
print(f"IMDb Vocabulary size: {len(imdb_vocab)}")
print(f"Sample IMDb words: {list(imdb_vocab)[:20]}")

**Reasoning**:
Following the natural progression of building an N-gram model, the next step after vocabulary creation is to count the unigrams from the new `imdb_train_corpus`.



In [ ]:
print("Counting unigrams for IMDb corpus...")
imdb_unigram_counts = count_unigrams(imdb_train_corpus)
print(f"Total IMDb unigrams: {sum(imdb_unigram_counts.values())}")
print(f"Unique IMDb unigrams: {len(imdb_unigram_counts)}")
print(f"\nMost common IMDb unigrams:")
print(imdb_unigram_counts.most_common(20))

**Reasoning**:
Following the natural progression of building an N-gram model, the next step after counting unigrams is to count bigrams from the new `imdb_train_corpus`.



In [ ]:
print("Counting bigrams for IMDb corpus...")
imdb_bigram_counts = count_bigrams(imdb_train_corpus)
print(f"Total IMDb bigrams: {sum(imdb_bigram_counts.values())}")
print(f"Unique IMDb bigrams: {len(imdb_bigram_counts)}")
print(f"\nMost common IMDb bigrams:")
print(imdb_bigram_counts.most_common(20))

**Reasoning**:
Following the natural progression of building an N-gram model, the next step after counting bigrams is to count trigrams from the new `imdb_train_corpus`.



In [ ]:
print("Counting trigrams for IMDb corpus...")
imdb_trigram_counts = count_trigrams(imdb_train_corpus)
print(f"Total IMDb trigrams: {sum(imdb_trigram_counts.values())}")
print(f"Unique IMDb trigrams: {len(imdb_trigram_counts)}")
print(f"\nMost common IMDb trigrams:")
print(imdb_trigram_counts.most_common(20))

**Reasoning**:
Now that we have counted the unigrams, bigrams, and trigrams for the IMDb corpus, the next step, following the original notebook's flow, is to implement and test the Unigram Model using this new data.



In [ ]:
print("Building Unigram Model for IMDb corpus...")
imdb_unigram_model = UnigramModel(imdb_train_corpus)

# Test your model
test_words_imdb = ["the", "a", "and", "<s>", "movie", "/><br"]
print("IMDb Unigram Probabilities:")
for word in test_words_imdb:
    prob = imdb_unigram_model.probability(word)
    print(f"  P({word}) = {prob:.6f}")

**Reasoning**:
Now that the Unigram Model for the IMDb corpus has been built and tested, the next logical step, following the original notebook's structure, is to implement and test the Bigram Model using the new `imdb_train_corpus`.



In [ ]:
print("Building Bigram Model for IMDb corpus...")
imdb_bigram_model = BigramModel(imdb_train_corpus)

# Test your model
test_bigrams_imdb = [("<s>", "the"), ("the", "movie"), ("of", "the"), ("/><br", "/>the")]
print("IMDb Bigram Probabilities:")
for w1, w2 in test_bigrams_imdb:
    prob = imdb_bigram_model.probability(w2, w1)
    print(f"  P({w2} | {w1}) = {prob:.6f}")

**Reasoning**:
Now that the Bigram Model for the IMDb corpus has been built and tested, the next logical step, following the original notebook's structure, is to implement and test the Trigram Model using the new `imdb_train_corpus`.



In [ ]:
print("Building Trigram Model for IMDb corpus...")
imdb_trigram_model = TrigramModel(imdb_train_corpus)

# Test your model
# Find a sample trigram from the corpus or create one that is likely to exist
# For simplicity, let's pick a common trigram we found earlier, e.g., ('one', 'of', 'the')
# or a trigram from the first sentence.

# Let's try to get a valid trigram from the first sentence if it's long enough
if len(imdb_train_corpus[0]) >= 3:
    w1, w2, w3 = imdb_train_corpus[0][0], imdb_train_corpus[0][1], imdb_train_corpus[0][2]
    # If the first word is '<s>', the actual context words start from index 1 and 2
    if w1 == '<s>' and len(imdb_train_corpus[0]) >= 4:
        w1, w2, w3 = imdb_train_corpus[0][1], imdb_train_corpus[0][2], imdb_train_corpus[0][3]

    prob = imdb_trigram_model.probability(w3, w2, w1)
    print(f"IMDb Trigram Probability: P({w3} | {w1}, {w2}) = {prob:.6f}")
else:
    print("First sentence in imdb_train_corpus is too short for trigram test.")

**Reasoning**:
Now that the N-gram models (Unigram, Bigram, Trigram) have been built for the IMDb corpus, the next step is to evaluate their performance using perplexity, similar to Part 6 of the original notebook. This requires preparing a test corpus from the IMDb dataset's 'test' split.



In [ ]:
print("Preparing IMDb test corpus...")
imdb_test_corpus = prepare_corpus(imdb_dataset['test'], max_examples=100)
print(f"IMDb Test corpus size: {len(imdb_test_corpus)} sentences")

print("\nCalculating IMDb model perplexities...")

imdb_unigram_perplexity = calculate_perplexity(imdb_unigram_model, imdb_test_corpus, model_type="unigram")
imdb_bigram_perplexity = calculate_perplexity(imdb_bigram_model, imdb_test_corpus, model_type="bigram")
imdb_trigram_perplexity = calculate_perplexity(imdb_trigram_model, imdb_test_corpus, model_type="trigram")

print(f"IMDb Unigram Model Perplexity: {imdb_unigram_perplexity:.2f}")
print(f"IMDb Bigram Model Perplexity: {imdb_bigram_perplexity:.2f}")
print(f"IMDb Trigram Model Perplexity: {imdb_trigram_perplexity:.2f}")

**Reasoning**:
The 'inf' perplexity for the unsmoothed unigram, bigram, and trigram models indicates that words or n-grams from the test set were not present in the training set, resulting in zero probabilities. To get meaningful perplexity scores, I will introduce add-one smoothing for unigram and trigram models, similar to the existing `BigramModelSmoothed`, and then use these smoothed models for evaluation.



In [ ]:
class UnigramModelSmoothed:
    """Unigram language model with add-one smoothing."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.total_words = sum(self.unigram_counts.values())
        self.vocab = build_vocabulary(corpus)
        self.vocab_size = len(self.vocab)

    def probability(self, word: str) -> float:
        """Calculate P(word) with add-one smoothing."""
        # P(word) = (C(word) + 1) / (N + V)
        numerator = self.unigram_counts.get(word, 0) + 1
        denominator = self.total_words + self.vocab_size
        return numerator / denominator if denominator > 0 else 0.0

    def log_probability(self, word: str) -> float:
        """Calculate log P(word)."""
        prob = self.probability(word)
        return np.log(prob) if prob > 0 else float('-inf')

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        log_prob_sum = 0.0
        for word in sentence:
            log_prob_sum += self.log_probability(word)
        return log_prob_sum


class TrigramModelSmoothed:
    """Trigram language model with add-one smoothing."""

    def __init__(self, corpus: List[List[str]]):
        self.bigram_counts = count_bigrams(corpus)
        self.trigram_counts = count_trigrams(corpus)
        self.vocab = build_vocabulary(corpus) # Need vocab size for smoothing
        self.vocab_size = len(self.vocab)

    def probability(self, word: str, prev_word1: str, prev_word2: str) -> float:
        """Calculate P(word | prev_word2, prev_word1) with add-one smoothing."""
        # P(wi | wi-2, wi-1) = (C(wi-2, wi-1, wi) + 1) / (C(wi-2, wi-1) + V)
        numerator = self.trigram_counts.get((prev_word2, prev_word1, word), 0) + 1
        denominator = self.bigram_counts.get((prev_word2, prev_word1), 0) + self.vocab_size

        return numerator / denominator if denominator > 0 else 0.0

    def log_probability(self, word: str, prev_word1: str, prev_word2: str) -> float:
        """Calculate log P(word | prev_word2, prev_word1)."""
        prob = self.probability(word, prev_word1, prev_word2)
        return np.log(prob) if prob > 0 else float('-inf')

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        log_prob_sum = 0.0
        if len(sentence) < 3:
            # For very short sentences, effectively probability is 1 (log prob 0) for initial tokens
            return 0.0

        for i in range(2, len(sentence)):  # Start from the third word (index 2)
            log_prob_sum += self.log_probability(sentence[i], sentence[i-1], sentence[i-2])
        return log_prob_sum

# Re-instantiate models with smoothing for perplexity calculation
print("Building Smoothed Unigram Model for IMDb corpus...")
imdb_unigram_model_smoothed = UnigramModelSmoothed(imdb_train_corpus)
print("Building Smoothed Bigram Model for IMDb corpus...")
imdb_bigram_model_smoothed = BigramModelSmoothed(imdb_train_corpus)
print("Building Smoothed Trigram Model for IMDb corpus...")
imdb_trigram_model_smoothed = TrigramModelSmoothed(imdb_train_corpus)

print("Preparing IMDb test corpus...")
imdb_test_corpus = prepare_corpus(imdb_dataset['test'], max_examples=100)
print(f"IMDb Test corpus size: {len(imdb_test_corpus)} sentences")

print("\nCalculating IMDb model perplexities with smoothing...")

imdb_unigram_perplexity = calculate_perplexity(imdb_unigram_model_smoothed, imdb_test_corpus, model_type="unigram")
imdb_bigram_perplexity = calculate_perplexity(imdb_bigram_model_smoothed, imdb_test_corpus, model_type="bigram")
imdb_trigram_perplexity = calculate_perplexity(imdb_trigram_model_smoothed, imdb_test_corpus, model_type="trigram")

print(f"IMDb Smoothed Unigram Model Perplexity: {imdb_unigram_perplexity:.2f}")
print(f"IMDb Smoothed Bigram Model Perplexity: {imdb_bigram_perplexity:.2f}")
print(f"IMDb Smoothed Trigram Model Perplexity: {imdb_trigram_perplexity:.2f}")

**Reasoning**:
The request asks to add a new step to apply the `prepare_corpus` function to the 'imdb' dataset's 'train' split, using `max_examples=1000`. This will generate the `imdb_train_corpus` and print its size and a sample.



In [ ]:
print("Preparing IMDb training corpus...")
imdb_train_corpus = prepare_corpus(imdb_dataset['train'], max_examples=1000)

print(f"\nIMDb Corpus size: {len(imdb_train_corpus)} sentences")
print(f"First IMDb sentence: {imdb_train_corpus[0][:10]}...")

**Reasoning**:
The request asks to add a new step to apply the `prepare_corpus` function to the 'imdb' dataset's 'train' split, using `max_examples=1000`. This will generate the `imdb_train_corpus` and print its size and a sample.



In [ ]:
print("Preparing IMDb training corpus...")
imdb_train_corpus = prepare_corpus(imdb_dataset['train'], max_examples=1000)

print(f"\nIMDb Corpus size: {len(imdb_train_corpus)} sentences")
print(f"First IMDb sentence: {imdb_train_corpus[0][:10]}...")

**Reasoning**:
The previous steps successfully loaded and processed the IMDb corpus, and evaluated smoothed n-gram models. The next logical step, following the original notebook's structure and the user's overall goal of completing bonus challenges, is to implement the `autocomplete` function from `Exercise 7.1: Implement Text Generator` (originally part 7, now revisited in bonus challenge 3) using a smoothed bigram model. This will provide an example of text generation and prediction.



In [ ]:
def autocomplete(partial_sentence: str, model: BigramModelSmoothed, k: int = 5) -> List[Tuple[str, float]]:
    """
    Suggest next words for autocomplete.

    Args:
        partial_sentence: Incomplete sentence
        model: Trained bigram model (should be a smoothed one for better results)
        k: Number of suggestions

    Returns:
        List of (word, probability) tuples
    """
    # Tokenize the partial sentence, ensuring we don't add <s> and </s> mid-sentence
    # We only care about the last word to predict the next.
    tokens = preprocess_text(partial_sentence, add_start_end=False)

    if not tokens:
        # If the partial sentence is empty, suggest words based on <s>
        current_word = '<s>'
    else:
        current_word = tokens[-1]

    suggestions = []
    # Iterate through all words in the model's vocabulary
    for next_word in model.vocab:
        # Exclude start and end tokens from suggestions unless specifically requested
        if next_word in ['<s>', '</s>']:
            continue

        prob = model.probability(next_word, current_word)
        if prob > 0:
            suggestions.append((next_word, prob))

    # Sort suggestions by probability in descending order
    suggestions.sort(key=lambda x: x[1], reverse=True)

    return suggestions[:k]

# Test
# Re-initialize the BigramGenerator with the smoothed model for text generation based on IMDb data.
# The existing BigramGenerator uses BigramModelSmoothed internally by default.
imdb_generator = BigramGenerator(imdb_train_corpus)

test_partial = "this movie is"
suggestions = autocomplete(test_partial, imdb_generator.bigram_model, k=5)
print(f"Suggestions for: '{test_partial}'")
for word, prob in suggestions:
    print(f"  {word}: {prob:.4f}")

test_partial_2 = "i love"
suggestions_2 = autocomplete(test_partial_2, imdb_generator.bigram_model, k=5)
print(f"\nSuggestions for: '{test_partial_2}'")
for word, prob in suggestions_2:
    print(f"  {word}: {prob:.4f}")


## Generate Sample Text (New Dataset)

### Subtask:
Use the `BigramGenerator` with the new smoothed bigram model to generate a few sample sentences from the new IMDb dataset to demonstrate its generative capabilities.


**Reasoning**:
To demonstrate the generative capabilities of the smoothed bigram model trained on the IMDb corpus, I will use the `imdb_generator` to produce five sample sentences and then print them, as requested by the subtask.



In [ ]:
print("Generated Sentences (IMDb Bigram Smoothed):")
print("=" * 60)
for i in range(5): # Generate 5 sentences
    sentence = imdb_generator.generate(max_length=20)
    print(f"{i+1}. {' '.join(sentence)}")

**Reasoning**:
The previous execution failed with a `NameError` because `imdb_generator` was not defined. This indicates that the cell initializing `imdb_generator` was either not executed or its state was lost. I need to re-initialize `imdb_generator` in this cell before attempting to use it.

